In [ ]:
"""#archive:
class My_Custom_Generator(keras.utils.Sequence) :
  
  def __init__(self, image_filenames, labels, batch_size) :
    self.image_filenames = image_filenames
    self.labels = labels
    self.batch_size = batch_size
    
    
  def __len__(self) :
    return (np.ceil(len(self.image_filenames) / float(self.batch_size))).astype(np.int)
  
  
  def __getitem__(self, idx) :
    batch_x = self.image_filenames[idx * self.batch_size : (idx+1) * self.batch_size]
    batch_y = self.labels[idx * self.batch_size : (idx+1) * self.batch_size]
    
    return np.array([
            resize(imread('/content/all_images/' + str(file_name)), (80, 80, 3))
               for file_name in batch_x])/255.0, np.array(batch_y)
"""

"""
inputs= [array2[:n],array3[:n],array4[:n],array1_4[:n],array1_5[:n],array1_6[:n]]
targets=[array1_1[:n],array1_2[:n],array1_3[:n]]


self.paddings=tf.constant([[0,0], [num_steps_out,0], [0,0]])
x = tf.slice(tf.pad(x,paddings=self.paddings),[0,0,0],x.shape)
        
"""

In [1]:
import tensorflow as tf
import keras
import numpy as np
import os
import sys

In [2]:
dataset_paths = [f'datasets/TM_dataset_enc9_part{i}.npz' for i in range(1,6)] 
batch_size=64
counter=0

In [ ]:
"""counter = 0
for k,dataset_path in enumerate(dataset_paths):
    print(f'dataset part {k} :')
    try :
        print('loading...')
        loaded = np.load(dataset_path) 
        array_drum = loaded['array1_drm']
        array_piano = loaded['array2_pia']
        array_gtr = loaded['array3_gtr']
        array_bass = loaded['array4_bss']

        num_samples = array_drum.shape[0]
        num_batches = int(num_samples/batch_size)
        print(f'\tnum samples: {num_samples}')
        print(f'\tnum batches: {num_batches}')

        for i in range(num_batches):
            counter+=1
            if counter%100==0:
                print(f'\t{counter} batches done')
            path=f'datasets/data_for_sequence/batch_{counter}'

            np.savez_compressed(
                path,
                array1_drm=array_drum[i*batch_size:(i+1)*batch_size],
                array2_pia=array_piano[i*batch_size:(i+1)*batch_size],
                array3_gtr=array_gtr[i*batch_size:(i+1)*batch_size],
                array4_bss=array_bass[i*batch_size:(i+1)*batch_size],
            )
    except:
        print(f'Error! {sys.exc_info()[0]} occurred')
        break
            
print()
print(f'{counter} batches created from {counter*batch_size} files')
"""

dataset part 0 :
	num samples: 122538
	num batches: 1914
	100 batches done
	200 batches done
	300 batches done
	400 batches done
	500 batches done
	600 batches done
	700 batches done
	800 batches done
	900 batches done
	1000 batches done
	1100 batches done
	1200 batches done
	1300 batches done
	1400 batches done
	1500 batches done
	1600 batches done
	1700 batches done
	1800 batches done
	1900 batches done
dataset part 1 :


In [ ]:
#number of files per subdataset:
    #part1: 122538 files, 1914 batches
    #part2: 116751 files, 1814 batches
    #part3: 101364 files, 1583 batches
    #part4: ca 109651 files, 1725 batches
    #part5: 106954 files, 1671 batches
# total batches 8707
# total files read 557248

In [3]:
counter=7036
n=5
print(f'dataset part {n} :')
print('loading...')
dataset_path=f'datasets/TM_dataset_enc9_part{n}.npz'

loaded = np.load(dataset_path) 
array_drum = loaded['array1_drm']
array_piano = loaded['array2_pia']
array_gtr = loaded['array3_gtr']
array_bass = loaded['array4_bss']

num_samples = array_drum.shape[0]
num_batches = int(num_samples/batch_size)
print(f'\tnum samples: {num_samples}')
print(f'\tnum batches: {num_batches}')

for i in range(num_batches):
    counter+=1
    if counter%100==0:
        print(f'\t{counter} batches done')
    path=f'datasets/data_for_sequence/batch_{counter}'

    np.savez_compressed(
        path,
        array1_drm=array_drum[i*batch_size:(i+1)*batch_size],
        array2_pia=array_piano[i*batch_size:(i+1)*batch_size],
        array3_gtr=array_gtr[i*batch_size:(i+1)*batch_size],
        array4_bss=array_bass[i*batch_size:(i+1)*batch_size],
    )
print(f'{counter} batches created from {counter*batch_size} files as of now')

dataset part 5 :
loading...
	num samples: 106954
	num batches: 1671
	7100 batches done
	7200 batches done
	7300 batches done
	7400 batches done
	7500 batches done
	7600 batches done
	7700 batches done
	7800 batches done
	7900 batches done
	8000 batches done
	8100 batches done
	8200 batches done
	8300 batches done
	8400 batches done
	8500 batches done
	8600 batches done
	8700 batches done
8707 batches created from 557248 files as of now


In [11]:
del loaded
del array_drum
del array_piano
del array_gtr
del array_bass

In [ ]:
class BatchDataGenerator(keras.utils.Sequence):
    def __init__(self,dataset_path,length,batch_size,pad=1):
        self.dataset_path=dataset_path
        self.length=length
        self.batch_size=batch_size
        self.paddings=tf.constant([[0,0], [pad,0], [0,0]])
    def __len__(self):
        return self.length
    def __getitem__(self,idx):
        loaded = np.load(str(self.dataset_path+'/batch_'+str(int(idx+1))))
        array1 = loaded['array1_drm']
        array2 = loaded['array2_pia']
        array3 = loaded['array3_gtr']
        array4 = loaded['array4_bss']
        dec_input = tf.slice(tf.pad(array1,paddings=self.paddings),[0,0,0],array1.shape)
        inputs = [
            array2,array3,array4,
            dec_input[:,:,:9],
            dec_input[:,:,9:18],
            dec_input[:,:,18:27]
        ]
        return inputs,array1

In [8]:
loaded = np.load('datasets/data_for_sequence/batch_2174.npz')
array1 = loaded['array1_drm']
array2 = loaded['array2_pia']
array3 = loaded['array3_gtr']
array4 = loaded['array4_bss']
print(array1.shape)
print(array2.shape)

(64, 128, 27)
(64, 128, 128)
